In [1]:
import pandas as pd
import tushare as ts
import os
import time, datetime
import threadpool
import StkDataKeeper as sk

def getDailyData():
    # create folder
    cwd = os.getcwd()
    # today_date = time.strftime("%Y%m%d")
    target_dir = "current" # today_date
    try:
        os.makedirs(name=target_dir)
    except:
        print ('folder already existed')

    # clean up the index with numbers, and make date the default index
    def pre_process(df):
        df.set_index('date', inplace=True)
        return df

    def obtain_daily_data(sk_code, target_dir = ""):
        
        # print ("Processing: "+sk_code)

        if target_dir!="":
            target_dir=target_dir+os.path.sep

        day_df = pre_process(ts.get_k_data(sk_code))
        csv_filename = target_dir + "day_" + sk_code + ".csv"
        day_df.to_csv(csv_filename)
        
        # print (day_df.head(1))

#         five_min_df = pre_process(ts.get_k_data(sk_code, ktype='5'))
#         csv_filename = target_dir + "5min_" + sk_code + ".csv"
#         five_min_df.to_csv(csv_filename)

#         print (five_min_df.head(1))

    # obtain stock list
    stock_basics = ts.get_stock_basics()
    code_series = stock_basics.index

    # prepare arguments
    args_list=[]
    for code in code_series:
        args_list.append(([code, target_dir], None))

    # collect
    pool = threadpool.ThreadPool(50)
    requests = threadpool.makeRequests(obtain_daily_data, args_list)
    [pool.putRequest(req) for req in requests]
    pool.wait()

    print ('Get data done')

In [2]:
while (True):
    now = datetime.datetime.now()
    today_begin1 = datetime.datetime(now.year, now.month, now.day, 9, 29)
    today_end1 = datetime.datetime(now.year, now.month, now.day, 11, 31)
    today_begin2 = datetime.datetime(now.year, now.month, now.day, 12, 59)
    today_end2 = datetime.datetime(now.year, now.month, now.day, 15, 1)

#     # create folder
#     cwd = os.getcwd()
#     today_date = time.strftime("%Y%m%d")
#     target_dir = cwd+os.path.sep+today_date+"_time"
#     try:
#         os.makedirs(name=target_dir)
#     except:
#         print ('folder already existed')

    now = datetime.datetime.now()

    duration_mins = 15

    while (now-today_end2).days < 0: 
        while (now.timetuple().tm_sec != 0) or (now.timetuple().tm_min % duration_mins != 0) :
            time.sleep(0.5)
            now = datetime.datetime.now()

        if (((today_begin1-now).days < 0 and (now-today_end1).days < 0) or ((today_begin2-now).days < 0 and (now-today_end2).days < 0)) :
            getDailyData()
            print(datetime.datetime.now())

        time.sleep(duration_mins * 60 - 300)
        now = datetime.datetime.now()
    
    time.sleep(3600)

KeyboardInterrupt: 

In [ ]:
getDailyData()